In [2]:
!pip install geopy


In [10]:
from geopy.geocoders import Nominatim
CLIENT_ID = 'JFGH3PREVZK0JAEXJ4WANVT0C1XNAHHUC4NHYTBBA2OR2P0W' # your Foursquare ID
CLIENT_SECRET = 'L1JMFW35UOM0TZBH3BGXQOT1DGTZ5OL5EIGQPJ11BTAMVHUY' # your Foursquare Secret
CLIENT_SECRET = 'L1JMFW35UOM0TZBH3BGXQOT1DGTZ5OL5EIGQPJ11BTAMVHUY' # your Foursquare Secret
VERSION = '20180604'
limit =50
radius=1000


In [11]:
geolocator=Nominatim(user_agent='foursquare_agent')
address="Klovervej 44 Kolding, Denmark, DK"
locator=geolocator.geocode(address)
latitude=locator.latitude
longitude=locator.longitude
print(latitude,longitude)

55.485765 9.497823


In [14]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=JFGH3PREVZK0JAEXJ4WANVT0C1XNAHHUC4NHYTBBA2OR2P0W&client_secret=L1JMFW35UOM0TZBH3BGXQOT1DGTZ5OL5EIGQPJ11BTAMVHUY&ll=55.485765,9.497823&v=20180604&radius=1000&limit=50'

In [15]:
import requests

response=requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5e4978983907e700247c4db5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kolding',
  'headerFullLocation': 'Kolding',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 55.49476500900001,
    'lng': 9.513677272866461},
   'sw': {'lat': 55.47676499099999, 'lng': 9.48196872713354}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c36e1e43849c928fac3bcb1',
       'name': 'Marina Syd',
       'location': {'address': 'Fjordstien',
        'lat': 55.487869206849695,
        'lng': 9.499942272170095,
        'labeledLatLngs': [{'label': 'display',
          'lat': 55.487869206849695,
          'lng': 9.4

In [37]:
n=len(response['response']['groups'][0]['items'])
print('There are ',n, 'attractions near ',address)

There are  5 attractions near  Klovervej 44 Kolding, Denmark, DK


In [41]:
for i in range(0,5):
    print(response['response']['groups'][0]['items'][i]['venue']['name'])

Marina Syd
REMA 1000
Kolding Kajak Klub
Kolding Havn
Fakta


In [45]:
from pandas.io.json import json_normalize
venues=json_normalize(response['response']['groups'][0]['items'])

In [46]:
venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c36e1e43849c928fac3bcb1-0,"[{'id': '4bf58dd8d48988d1e0941735', 'name': 'H...",4c36e1e43849c928fac3bcb1,Fjordstien,DK,Kolding,Danmark,NaN,269,"[Fjordstien, 6000 Kolding, Danmark]","[{'label': 'display', 'lat': 55.48786920684969...",55.487869,9.499942,6000,Region Syddanmark,Marina Syd,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d21e3da6e8c3704f8b906a0-1,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",4d21e3da6e8c3704f8b906a0,"Kløvervej 39, Tved",DK,Kolding,Danmark,Tvedvej,304,"[Kløvervej 39, Tved (Tvedvej), 6000 Kolding, D...","[{'label': 'display', 'lat': 55.48418432471778...",55.484184,9.501754,6000,Region Syddanmark,REMA 1000,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-51bc50af498e9a4f34359b32-2,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",51bc50af498e9a4f34359b32,NaN,DK,Kolding,Danmark,NaN,833,"[Kolding, Danmark]","[{'label': 'display', 'lat': 55.48831435702992...",55.488314,9.485405,NaN,Region Syddanmark,Kolding Kajak Klub,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8edd6a61afe70d47cf3346-3,"[{'id': '4bf58dd8d48988d1e0941735', 'name': 'H...",4e8edd6a61afe70d47cf3346,Jens Holms Vej 1,DK,Kolding,Danmark,NaN,892,"[Jens Holms Vej 1, 6000 Kolding, Danmark]","[{'label': 'display', 'lat': 55.49083733965614...",55.490837,9.486867,6000,Region Syddanmark,Kolding Havn,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb6f0b9589f236a34e53a3d-4,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",4cb6f0b9589f236a34e53a3d,Agtrupvej,DK,Kolding,Danmark,NaN,891,"[Agtrupvej, 6000 Kolding, Danmark]","[{'label': 'display', 'lat': 55.47883716192091...",55.478837,9.490720,6000,Region Syddanmark,Fakta,0,[]


In [47]:
venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups'],
      dtype='object')

In [48]:
venues.drop(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs', 'venue.location.postalCode',
       'venue.location.state','venue.photos.count',
       'venue.photos.groups'],axis=1,inplace=True)

In [50]:
venues

,venue.location.lat,venue.location.lng,venue.name
0,55.487869,9.499942,Marina Syd
1,55.484184,9.501754,REMA 1000
2,55.488314,9.485405,Kolding Kajak Klub
3,55.490837,9.486867,Kolding Havn
4,55.478837,9.490720,Fakta


In [57]:
import folium

map=folium.Map([latitude,longitude],zoom_start=13)

folium.CircleMarker([latitude,longitude],
                   radius=5,
                   fill=True,
                   color='black',
                   fillcolor='green',
                   popup='Metropolis').add_to(map)
for lat,long,label in zip(venues['venue.location.lat'],venues['venue.location.lng'],venues['venue.name']):
    folium.CircleMarker(
                        [lat,long],
                        radius=5,
                        fill=True,
                        color='black',
                        fill_color='green',
                        fill_opacity=0.4,
                        popup=label
    ).add_to(map)
    
map